In [138]:
import pandas as pd
import numpy as np
import os
from os import pardir, path
import sys
import warnings
warnings.filterwarnings('ignore')

from sklearn.linear_model import LinearRegression, Ridge, Lasso
from sklearn import metrics

from sklearn import ensemble
from sklearn.model_selection import GridSearchCV, RandomizedSearchCV

In [139]:
mod_path = os.getcwd()
if mod_path not in sys.path:
    sys.path.append(mod_path)
print(sys.path)

from func import Model

['/Users/karimkhalil/Coding/development/commodity', '/Users/karimkhalil/opt/anaconda3/lib/python39.zip', '/Users/karimkhalil/opt/anaconda3/lib/python3.9', '/Users/karimkhalil/opt/anaconda3/lib/python3.9/lib-dynload', '', '/Users/karimkhalil/opt/anaconda3/lib/python3.9/site-packages', '/Users/karimkhalil/opt/anaconda3/lib/python3.9/site-packages/aeosa']


In [140]:
## mapping

mat_codes = {
    "F": "01",
    "G": "02", 
    "H": "03", 
    "J": "04", 
    "K": "05",
    "M": "06",
    "N": "07", 
    "Q": "08",
    "U" : "09",
    "V": "10",
    "X": "11",
    "Z": "12"
}

## 1. Data Preparation

In [141]:
df = pd.read_csv('commodities.csv')
df['datetime'] = pd.to_datetime(df['date'], utc=True)
df['date_strf']=df.datetime.dt.strftime('%Y%m%d')
df['dayofweek'] = df['datetime'].dt.strftime("%a")
df['month'] = df['datetime'].dt.month

df['datetime_maturity'] = pd.to_datetime(df.maturity.str[-4:] + df.maturity.str[0].map(mat_codes), format='%Y%m', utc=True)

df['datetime_maturity'] = df['datetime_maturity']-pd.Timedelta(1, "d")
df['date_strf_maturity']=df.datetime_maturity.dt.strftime('%Y%m%d')
df['time2maturity_d'] = (df.datetime_maturity-df.datetime).dt.days

df.loc[df['time2maturity_d'] <0, 'time2maturity_d'] = 0

df['time2maturity_m'] = (df.time2maturity_d/30).round()
df_settle = df.loc[df['observation'] == 'Settle']
df_soy = df_settle.loc[df_settle['instrument'] == 'CBOT.ZS']
df_soy.sort_values(['datetime' , 'datetime_maturity'], ascending = [True, False], inplace=True)




In [142]:
## select prices with only 6 months maturity for comparability

dates = set(df_soy['date_strf'])

concat = []

for i in dates:
    duration = 6
    data = df_soy.loc[(df_soy['date_strf'] == i) & (df_soy['time2maturity_m'] == duration)]
    while data.shape[0] ==0:
        duration +=1
        data = df_soy.loc[(df_soy['date_strf'] == i) & (df_soy['time2maturity_m'] == duration)]
    concat.append(data)

df_soy_6m = pd.concat(concat)
df_soy_6m.set_index('date_strf' , inplace=True, drop=True)
df_soy_6m.sort_index(inplace=True)

### Create Features

In [143]:
## calculate returns for the previous 7 days

for i in range(7):
    df_soy_6m[f'pct_t-{i+1}'] = df_soy_6m.value.pct_change(i+1)

## rolling averages for prices and returns

vals = ['value', 'pct_t-1']

for i in [7, 15, 30 , 60]:
    for j in vals:
        df_soy_6m[f'roll_avg_pct_{i}'] = df_soy_6m[j].rolling(i).mean()

for i in vals:
    df_soy_6m[f'exp_avg_{i}'] = df_soy_6m[i].expanding(1).mean()

In [144]:
df_soy_6m[[i for i in df_soy_6m.columns if df_soy_6m[i].dtype != 'datetime64[ns, UTC]']].to_excel(os.path.join(os.getcwd(), 'df_soy_6m.xlsx'))

In [145]:
## drop missing values
df_soy_6m_clean = df_soy_6m.dropna()

In [146]:
desc = df_soy_6m_clean.describe()
desc

,value,month,time2maturity_d,time2maturity_m,pct_t-1,pct_t-2,pct_t-3,pct_t-4,pct_t-5,pct_t-6,pct_t-7,roll_avg_pct_7,roll_avg_pct_15,roll_avg_pct_30,roll_avg_pct_60,exp_avg_value,exp_avg_pct_t-1
count,171.000000,171.000000,171.000000,171.000000,171.000000,171.000000,171.000000,171.000000,171.000000,171.000000,171.000000,171.000000,171.000000,171.000000,171.000000,171.000000,171.000000
mean,1320.261696,7.508772,194.941520,6.508772,0.000423,0.000711,0.001131,0.001545,0.001937,0.002345,0.002804,0.000419,0.000242,0.000150,0.000047,1315.436391,0.000286
std,55.533812,2.324722,17.504187,0.535432,0.015322,0.021192,0.026772,0.030715,0.033647,0.035290,0.036951,0.005274,0.002823,0.001888,0.001142,12.715297,0.000377
min,1212.000000,3.000000,165.000000,6.000000,-0.066481,-0.086664,-0.100876,-0.127560,-0.139802,-0.133126,-0.138179,-0.020777,-0.007644,-0.003393,-0.001890,1286.934932,-0.000457
25%,1273.125000,6.000000,180.000000,6.000000,-0.006777,-0.010777,-0.016196,-0.017538,-0.018498,-0.021231,-0.021902,-0.003106,-0.001641,-0.001278,-0.000881,1311.638898,0.000012
50%,1308.750000,8.000000,195.000000,6.000000,0.001292,0.001661,0.001881,0.000965,-0.000396,0.000918,0.000000,0.000063,-0.000108,-0.000211,-0.000122,1320.301802,0.000179
75%,1356.375000,9.500000,210.000000,7.000000,0.007343,0.013360,0.016537,0.020027,0.020845,0.022471,0.025343,0.003619,0.002086,0.001139,0.001105,1325.244961,0.000495
max,1461.000000,12.000000,225.000000,8.000000,0.063438,0.065332,0.098921,0.097939,0.100883,0.087014,0.079443,0.011018,0.007952,0.006661,0.002231,1328.049051,0.001290


In [147]:
def return_tot(df, date, value):
    period_beg = df.date.min()
    period_end = df.date.max()

    price_beg = df.loc[df.date == period_beg, value].values[0]
    price_end = df.loc[df.date == period_end, value].values[0]

    ret = (price_end-price_beg)/ price_beg
    return ret

sharpe = desc.loc['mean', 'pct_t-1'] / desc.loc['std', 'pct_t-1']
avg_return = desc.loc['mean', 'pct_t-1']
tot_return = return_tot(df_soy_6m, 'date', 'value')

print(f'Sharpe ratio: {100*(sharpe):.2f} %')
print(f'Average daily return: {100*(avg_return):.2f} %')
print(f'Total return: {100*(tot_return):.2f} %')

Sharpe ratio: 2.76 %
Average daily return: 0.04 %
Total return: -4.12 %


In [148]:
import plotly.figure_factory as ff
import plotly.express as px
from plotly.subplots import make_subplots
import plotly.graph_objects as go

df_sorted = df_soy_6m_clean.sort_values('date')
fig = make_subplots(rows=2, cols=2, subplot_titles=["Price", "Daily Return Distribution", 'Price Distribution', 'Return Distribution'])


dist = ff.create_distplot([df_soy_6m_clean['pct_t-1'].values.tolist()], [''], bin_size=.01).data[1]
hist = ff.create_distplot([df_soy_6m_clean['pct_t-1'].values.tolist()], [''], bin_size=.01).data[0]
price = px.line(x=df_soy_6m_clean.value, y=df_soy_6m_clean.index)
price = px.line(y=df_sorted.value, x=df_sorted.date, title='Bean Price', labels=dict(x='Date', y='USD')).data[0]
# dist.update_layout(width=700, height=700)

fig.add_trace(go.Scatter(dist, line=dict(color='red')), row=1, col=2)
fig.add_trace(hist, row=1, col=2)
fig.add_trace(go.Scatter(price), row=1,col=1)
fig.add_trace(go.Box(x=df_sorted.value), row=2,col=1)
fig.add_trace(go.Box(x=df_sorted['pct_t-1']), row=2,col=2)

fig['layout'].update(height=800, width=1500, title='Returns Descriptive Statistics')

fig.update_xaxes(title_text="Date", row=1, col=1)
fig.update_yaxes(title_text="USD", row=1, col=1)

# fig.add_trace(dist, row=1, col=1)
fig.show()

In [149]:
model = Model(df_soy_6m_clean)

object instanciated


In [150]:
l = model.train(df = model.df, col_period='date' , train_window=10 , test_window=1, test_gap = 0, expanding=False)

In [151]:
regr = LinearRegression()
ridge_regr = Ridge()
xgb = ensemble.GradientBoostingRegressor()
histgrad = ensemble.HistGradientBoostingRegressor()
randforest = ensemble.RandomForestRegressor()

In [152]:
df_soy_6m.columns

Index(['date', 'instrument', 'maturity', 'observation', 'value', 'currency',
       'datetime', 'dayofweek', 'month', 'datetime_maturity',
       'date_strf_maturity', 'time2maturity_d', 'time2maturity_m', 'pct_t-1',
       'pct_t-2', 'pct_t-3', 'pct_t-4', 'pct_t-5', 'pct_t-6', 'pct_t-7',
       'roll_avg_pct_7', 'roll_avg_pct_15', 'roll_avg_pct_30',
       'roll_avg_pct_60', 'exp_avg_value', 'exp_avg_pct_t-1'],
      dtype='object')

In [153]:
cols_x = ['pct_t-2', 'pct_t-3', 'pct_t-4', 'pct_t-5', 'pct_t-6', 'pct_t-7','roll_avg_pct_7', 'roll_avg_pct_15', 'roll_avg_pct_30']
cols_y = 'pct_t-1'

In [154]:
# model.skpredict(df_soy_6m, df_soy_6m, xgb, cols_x, cols_y, printstat=True)

In [155]:
df_result = model.skpredict_window(xgb, cols_x, cols_y, 'date' , train_window=4 , test_window=1, test_gap = 0, expanding=False, print_iter=False)[1]

Average MSE train: 1.1599378044768752e-13
Average MSE test: 0.0003266726921681468


In [191]:
df_result = model.skpredict_window(randforest, cols_x, cols_y, 'date' , train_window=4 , test_window=1, test_gap = 0, expanding=False, print_iter=False)[1]

Average MSE train: 3.3224788908657286e-05
Average MSE test: 0.00027879693183543297


In [ ]:
for i in range(4,30,2):
    print(i)
    df_result = model.skpredict_window(randforest, cols_x, cols_y, 'date' , train_window=i , test_window=1, test_gap = 0, expanding=True, print_iter=False)[1]

4
Average MSE train: 3.664851877381279e-05
Average MSE test: 0.0001934246374458603
6
Average MSE train: 3.668222394897002e-05
Average MSE test: 0.00019098456835635037
8
Average MSE train: 3.685788244555677e-05
Average MSE test: 0.00018450609326009387
10
Average MSE train: 3.659513886241697e-05
Average MSE test: 0.00018271488608053436
12
Average MSE train: 3.655076567347889e-05
Average MSE test: 0.00018423117482954378
14
Average MSE train: 3.625201306686981e-05
Average MSE test: 0.00019164529807225015
16
Average MSE train: 3.6663929287039285e-05
Average MSE test: 0.00019803652268188927
18
Average MSE train: 3.659857029958739e-05
Average MSE test: 0.00019248563476372514
20
Average MSE train: 3.647705756684366e-05
Average MSE test: 0.00019865863210750997
22
Average MSE train: 3.653372739556396e-05
Average MSE test: 0.00019366528485573227
24
Average MSE train: 3.6600498042780664e-05
Average MSE test: 0.00019619531457240537
26
Average MSE train: 3.6851693189664384e-05
Average MSE test: 0.00

In [ ]:
for i in range(4,30,2):
    print(i)
    df_result = model.skpredict_window(randforest, cols_x, cols_y, 'date' , train_window=i , test_window=1, test_gap = 0, expanding=False, print_iter=False)[1]


4
Average MSE train: 3.415133269443157e-05
Average MSE test: 0.00027819979040408243
6
Average MSE train: 3.500574333702182e-05
Average MSE test: 0.00028424020542921786
8
Average MSE train: 3.516461100779567e-05
Average MSE test: 0.0002593273959717442
10
Average MSE train: 3.711933311653708e-05
Average MSE test: 0.0002549084897402791
12
Average MSE train: 3.853961602581963e-05
Average MSE test: 0.0002333023304870919
14
Average MSE train: 3.4988795978788745e-05
Average MSE test: 0.00022418790848416261
16
Average MSE train: 3.462219052644725e-05
Average MSE test: 0.00022345744135907343
18
Average MSE train: 3.351522968697158e-05
Average MSE test: 0.0002138777062939376
20
Average MSE train: 3.23287265431366e-05
Average MSE test: 0.0002244999553034259
22
Average MSE train: 3.267298443126664e-05
Average MSE test: 0.00021087911055340952
24
Average MSE train: 3.272597737202677e-05
Average MSE test: 0.00021437303377784935
26
Average MSE train: 3.219428896445834e-05
Average MSE test: 0.000202779

In [ ]:
df_result = model.skpredict_window(randforest, cols_x, cols_y, 'date' , train_window=16 , test_window=1, test_gap = 0, expanding=True, print_iter=False)[1]
df_result['position'] = df_result.apply(lambda row: row['pct_t-1'] if row['predict'] > 0 else -row['predict']  ,axis = 1)

Average MSE train: 3.6388460135350874e-05
Average MSE test: 0.00019162708859369115


In [264]:
df_test = df_result[['value', 'pct_t-1' , 'predict', 'MSE']]
df_test.loc[df_test['predict'] > 0, 'strat1_return'] = df_test['pct_t-1']
df_test.loc[df_test['predict'] < 0, 'strat1_return'] = -df_test['pct_t-1']

df_test.sort_index(inplace=True)
df_test['validate'] = df_test['value'].shift(1) * (1+df_test['pct_t-1'])
df_test

,value,pct_t-1,predict,MSE,strat1_return,validate
date_strf,,,,,,
20210407,1270.75,-0.000393,0.012280,1.606165e-04,-0.000393,NaN
20210408,1273.25,0.001967,0.001430,2.882543e-07,0.001967,1273.25
20210409,1263.25,-0.007854,0.001695,9.117571e-05,-0.007854,1263.25
20210412,1250.25,-0.010291,-0.003207,5.017842e-05,0.010291,1250.25
20210413,1251.25,0.000800,-0.007324,6.599104e-05,-0.000800,1251.25
...,...,...,...,...,...,...
20211123,1299.25,-0.001729,-0.000266,2.139021e-06,0.001729,1299.25
20211124,1292.25,-0.005388,0.000728,3.739745e-05,-0.005388,1292.25
20211126,1278.75,-0.010447,-0.001490,8.021968e-05,0.010447,1278.75


In [361]:
def backtest(df, value, strat_return, pct_change, predict_return):
    ## produce array for t and t-1 indexing
    
    period_arr = np.empty([0,2])
    period_idx = [i for i in df.index]

    for i,j in enumerate(period_idx):
        if i ==0:
            x, y = j, np.nan
        else:
            x,y = j, period_idx[i-1]
        arr = np.array([[x,y]])
        period_arr = np.append(period_arr, arr, axis=0)

    ## back test results of strategy
    price_beg = df.loc[period_arr[0,0]][value]
    for i in df.index:
        period_prev = period_arr[period_arr[:,0] ==i][:,1][0]
        period_now = i
        price_now = df.loc[i][value]
        return_predict = df.loc[i][strat_return]
        predict = df.loc[i][predict_return]
        return_act = df.loc[i][pct_change]

        if i == df.index.min():
            pass
        
        else:
            price_prev = df.loc[period_prev][value]
            return_strat = return_predict * price_beg
            price_end = price_beg + return_strat
            # print(i, price_prev, price_now, return_act, return_predict, predict, return_strat ,price_beg, price_end)
            df.loc[i, 'value_strat'] = price_end
            price_beg = price_end
    return df
    
backtest(df_test, 'value' , 'strat1_return' ,'pct_t-1' , 'predict')

,value,pct_t-1,predict,MSE,strat1_return,validate,value_strat
date_strf,,,,,,,
20210407,1270.75,-0.000393,0.012280,1.606165e-04,-0.000393,NaN,NaN
20210408,1273.25,0.001967,0.001430,2.882543e-07,0.001967,1273.25,1273.250000
20210409,1263.25,-0.007854,0.001695,9.117571e-05,-0.007854,1263.25,1263.250000
20210412,1250.25,-0.010291,-0.003207,5.017842e-05,0.010291,1250.25,1276.250000
20210413,1251.25,0.000800,-0.007324,6.599104e-05,-0.000800,1251.25,1275.229204
...,...,...,...,...,...,...,...
20211123,1299.25,-0.001729,-0.000266,2.139021e-06,0.001729,1299.25,1728.851001
20211124,1292.25,-0.005388,0.000728,3.739745e-05,-0.005388,1292.25,1719.536430
20211126,1278.75,-0.010447,-0.001490,8.021968e-05,0.010447,1278.75,1737.500246


In [360]:
def price_predict(df):
    period_arr = np.empty([0,2])
    period_idx = [i for i in df.index]

    for i,j in enumerate(period_idx):
        if i ==0:
            x, y = j, np.nan
        else:
            x,y = j, period_idx[i-1]
        arr = np.array([[x,y]])
        period_arr = np.append(period_arr, arr, axis=0)

    
    return period_arr

period_arr=price_predict(df_test)
# period_arr

In [267]:
df_test.loc['20210426']['value']

1362.0

In [331]:
period_arr[1,0]

'20210408'

In [345]:
period_arr[1]

array(['20210408', '20210407'], dtype='<U32')

In [356]:
def value_strat(df, value, strat_return, pct_change, predict_return):
    price_beg = df.loc[period_arr[0,0]][value]
    for i in df.index:
        period_prev = period_arr[period_arr[:,0] ==i][:,1][0]
        period_now = i
        price_now = df.loc[i][value]
        return_predict = df.loc[i][strat_return]
        predict = df.loc[i][predict_return]
        return_act = df.loc[i][pct_change]

        if i == df.index.min():
            pass
        
        else:
            price_prev = df.loc[period_prev][value]
            return_strat = return_predict * price_beg
            price_end = price_beg + return_strat
            # print(i, price_prev, price_now, return_act, return_predict, predict, return_strat ,price_beg, price_end)
            df.loc[i, 'value_strat'] = price_end
            price_beg = price_end
    return df

value_strat(df_test, 'value' , 'strat1_return' ,'pct_t-1' , 'predict')


,value,pct_t-1,predict,MSE,strat1_return,validate,value_strat
date_strf,,,,,,,
20210407,1270.75,-0.000393,0.012280,1.606165e-04,-0.000393,NaN,NaN
20210408,1273.25,0.001967,0.001430,2.882543e-07,0.001967,1273.25,1273.250000
20210409,1263.25,-0.007854,0.001695,9.117571e-05,-0.007854,1263.25,1263.250000
20210412,1250.25,-0.010291,-0.003207,5.017842e-05,0.010291,1250.25,1276.250000
20210413,1251.25,0.000800,-0.007324,6.599104e-05,-0.000800,1251.25,1275.229204
...,...,...,...,...,...,...,...
20211123,1299.25,-0.001729,-0.000266,2.139021e-06,0.001729,1299.25,1728.851001
20211124,1292.25,-0.005388,0.000728,3.739745e-05,-0.005388,1292.25,1719.536430
20211126,1278.75,-0.010447,-0.001490,8.021968e-05,0.010447,1278.75,1737.500246


In [353]:

price_beg = df_test.loc[period_arr[0,0]]['value']
print(price_beg)
for i in df_test.index:
    period_prev = period_arr[period_arr[:,0] ==i][:,1][0]
    period_now = i
    price_now = df_test.loc[i]['value']
    return_predict = df_test.loc[i]['strat1_return']
    predict = df_test.loc[i]['predict']
    return_act = df_test.loc[i]['pct_t-1']

    if i == df_test.index.min():
        pass
    
    else:
        price_prev = df_test.loc[period_prev]['value']
        return_strat = return_predict * price_beg
        price_end = price_beg + return_strat

        print(i, price_prev, price_now, return_act, return_predict, predict, return_strat ,price_beg, price_end)
        df_test.loc[i, 'value_strat'] = price_end
        price_beg = price_end
    # df_test.loc['20210423']['predict']

1270.75
20210408 1270.75 1273.25 0.001967342120794857 0.001967342120794857 0.0014304489718270008 2.500000000000065 1270.75 1273.25
20210409 1273.25 1263.25 -0.007853917141174116 -0.007853917141174116 0.0016946800579410292 -9.999999999999943 1273.25 1263.25
20210412 1263.25 1250.25 -0.010290916287354013 0.010290916287354013 -0.003207243450131447 12.999999999999957 1263.25 1276.25
20210413 1250.25 1251.25 0.0007998400319935151 -0.0007998400319935151 -0.007323646810334848 -1.0207958408317235 1276.25 1275.2292041591684
20210414 1251.25 1264.0 0.010189810189810133 -0.010189810189810133 -0.002991157426365405 -12.99434353888456 1275.2292041591684 1262.2348606202838
20210415 1264.0 1269.0 0.003955696202531556 0.003955696202531556 0.005364150468674583 4.993017644858604 1262.2348606202838 1267.2278782651424
20210416 1269.0 1274.0 0.003940110323088941 0.003940110323088941 0.005209956637324254 4.9930176448585835 1267.2278782651424 1272.220895910001
20210419 1274.0 1284.0 0.007849293563579218 0.007

In [354]:
df_test

,value,pct_t-1,predict,MSE,strat1_return,validate,value_strat
date_strf,,,,,,,
20210407,1270.75,-0.000393,0.012280,1.606165e-04,-0.000393,NaN,NaN
20210408,1273.25,0.001967,0.001430,2.882543e-07,0.001967,1273.25,1273.250000
20210409,1263.25,-0.007854,0.001695,9.117571e-05,-0.007854,1263.25,1263.250000
20210412,1250.25,-0.010291,-0.003207,5.017842e-05,0.010291,1250.25,1276.250000
20210413,1251.25,0.000800,-0.007324,6.599104e-05,-0.000800,1251.25,1275.229204
...,...,...,...,...,...,...,...
20211123,1299.25,-0.001729,-0.000266,2.139021e-06,0.001729,1299.25,1728.851001
20211124,1292.25,-0.005388,0.000728,3.739745e-05,-0.005388,1292.25,1719.536430
20211126,1278.75,-0.010447,-0.001490,8.021968e-05,0.010447,1278.75,1737.500246


In [219]:
arr = np.empty([0,2])

# for i in range(3):
#     arr = np.append(arr, np.array([[10.1,20,30]]), axis=0)

arr

array([], shape=(0, 2), dtype=float64)

array([['20210423', 'nan'],
       ['20210426', '20210423'],
       ['20210427', '20210426'],
       ['20210428', '20210427'],
       ['20210429', '20210428'],
       ['20210430', '20210429'],
       ['20210503', '20210430'],
       ['20210504', '20210503'],
       ['20210505', '20210504'],
       ['20210506', '20210505'],
       ['20210507', '20210506'],
       ['20210510', '20210507'],
       ['20210511', '20210510'],
       ['20210512', '20210511'],
       ['20210513', '20210512'],
       ['20210514', '20210513'],
       ['20210517', '20210514'],
       ['20210518', '20210517'],
       ['20210519', '20210518'],
       ['20210520', '20210519'],
       ['20210521', '20210520'],
       ['20210524', '20210521'],
       ['20210525', '20210524'],
       ['20210526', '20210525'],
       ['20210527', '20210526'],
       ['20210528', '20210527'],
       ['20210601', '20210528'],
       ['20210602', '20210601'],
       ['20210603', '20210602'],
       ['20210604', '20210603'],
       ['202106

In [232]:
period_arr[:,0]

array(['20210423', '20210426', '20210427', '20210428', '20210429',
       '20210430', '20210503', '20210504', '20210505', '20210506',
       '20210507', '20210510', '20210511', '20210512', '20210513',
       '20210514', '20210517', '20210518', '20210519', '20210520',
       '20210521', '20210524', '20210525', '20210526', '20210527',
       '20210528', '20210601', '20210602', '20210603', '20210604',
       '20210607', '20210608', '20210609', '20210610', '20210611',
       '20210614', '20210615', '20210616', '20210617', '20210618',
       '20210621', '20210622', '20210623', '20210624', '20210625',
       '20210628', '20210629', '20210630', '20210701', '20210702',
       '20210706', '20210707', '20210708', '20210709', '20210712',
       '20210713', '20210714', '20210715', '20210716', '20210719',
       '20210720', '20210721', '20210722', '20210723', '20210726',
       '20210727', '20210728', '20210729', '20210730', '20210802',
       '20210803', '20210804', '20210805', '20210806', '202108

In [58]:
(1304.00 - 1323.00)/1323.00

-0.01436130007558579